In [8]:
# ============================================================================
# CONFIGURATION
# This cell contains all the settings a user needs to change.
# ============================================================================

# --- PATHS ---
# TODO: Update this path to point to your complete, "ground truth" data file.
INPUT_FILE_PATH = "data/Ground_Truth_Imputed.xlsx"

# The directory where the new datasets with missing values will be saved.
OUTPUT_DIR = "output/ArtificialMissingness"

# --- PARAMETERS ---
# The overall proportion of missing values to introduce into the data.
MISSING_RATE = 0.15

# For the MAR pattern, which column should be used to determine missingness in others?
# If set to None, the script will automatically use the first numeric column.
MAR_CONDITIONAL_COLUMN = None  # or e.g., "Metabolite_X"

Output will be saved to: '/Users/aranpurdy/Desktop/CFPS/ArtificialMissingness_numpy'
Found 40 numeric columns to target for amputation.

Introducing MCAR pattern...
  Completed in 0.01 seconds.
MCAR data saved successfully.

Introducing MAR pattern...
Using 'TM2A2_1' as the conditional column for MAR.
  Completed in 0.02 seconds.
MAR data saved successfully.

Introducing MNAR pattern...
  Completed in 0.03 seconds.
MNAR data saved successfully.

Script finished.
